# Data Analysis for Santa Clara Master Gardener Club by Michelle Petersen

In [ ]:
# Imports
%matplotlib inline
import pandas as pd
import numpy as np
import json
import geojson
import shapefile
import requests
from bs4 import BeautifulSoup
import pprint as pp
import matplotlib.pyplot as plt
from timeit import default_timer as timer

## Gather

#### Read Master Gardener Contacts and Nextdoor Neighborhood Information from Google Doc Spreadsheet

In [ ]:
import gspread
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession

# The scope to read the spreadsheet
_GS_SCOPE = ['https://www.googleapis.com/auth/spreadsheets.readonly',
            'https://www.googleapis.com/auth/drive']

# The ID and range of the MasterGardener spreadsheet.
_GS_ID = '1pPOuRZ9YsKwXGKgxzSm9tXCR_dgkCY4WNh3rg_cFX08'

service_account_info = json.load(open('Master Gardener-7e906e507d9d.json'))
credentials = service_account.Credentials.from_service_account_info(
    service_account_info,
    scopes=_GS_SCOPE)

if credentials:
    gc = gspread.Client(auth=credentials)
    gc.session = AuthorizedSession(credentials)
    book = gc.open_by_key(_GS_ID)
    print(book.title)
    worksheet_list = book.worksheets()
    for worksheet in worksheet_list:
        print(worksheet)

In [ ]:
# Load the data from each sheet into a dataframe
df_contacts = pd.DataFrame(book.get_worksheet(0).get_all_values())
df_contact_neighborhood = pd.DataFrame(book.get_worksheet(1).get_all_values())
df_nextdoor_neighborhoods = pd.DataFrame(book.get_worksheet(2).get_all_values())

In [ ]:
# Update the column names for the contact worksheet
df_contacts.reset_index()
df_contacts.columns = ['Name', 'Email', 'Neighborhood', 'State', 'County', 'City']

In [ ]:
# Remove the row with column names since gspread reads it as data
df_contacts = df_contacts[df_contacts['Name'] != 'Name']

In [ ]:
# Verify column names and data
df_contacts.head()

In [ ]:
# Update the column names for the contact to neighborhood mapping worksheet
df_contact_neighborhood.reset_index()
df_contact_neighborhood.columns = ['Name', 'Neighborhood', 'State', 'County', 'City']

In [ ]:
# Remove the row with column names since gspread reads it as data
df_contact_neighborhood = df_contact_neighborhood[df_contact_neighborhood['Name'] != 'Name']

In [ ]:
# Verify column names and data
df_contact_neighborhood.head()

### Store

In [ ]:
# Resave information to csv files for input to Tableau
df_contacts.to_csv('mg_contacts_master.csv', index=False)
df_contact_neighborhood.to_csv('mg_contact_neighborhood_master.csv', index=False)

In [ ]:
# Update the column names for the nextdoor neighborhood worksheet
df_nextdoor_neighborhoods.reset_index()
df_nextdoor_neighborhoods.columns = ['State', 'County', 'City', 'Neighborhood', 'Link', 'Nextdoor ID', 
                                     'Geometry', 'Interests', 'Interest in Gardening', 'Percentage of Homeowners', 
                                     'Number of Residents', 'Average Age', 'Attributes']

In [ ]:
# Remove the row with column names since gspread reads it as data
df_nextdoor_neighborhoods = df_nextdoor_neighborhoods[df_nextdoor_neighborhoods['Neighborhood'] != 'Neighborhood']

In [ ]:
# Verify column names and data
df_nextdoor_neighborhoods.head()

In [ ]:
# Verify neighborhood columns and data
df_nextdoor_neighborhoods.info()

In [ ]:
# Run some stats
df_nextdoor_neighborhoods.describe()

In [ ]:
# Resave neighborhood information to csv files for input to Tableau
df_nextdoor_neighborhoods.to_csv('nextdoor_with_properties_master.csv', index=False)

In [ ]:
# Save geographic information to geojson file for input to Tableau
from geojson import Point, Feature, FeatureCollection, dumps

features = []
for index, row in df_nextdoor_neighborhoods.iterrows():
    if (len(row["Geometry"]) > 0):
        features.append(json.loads(row["Geometry"]))

feature_collection = FeatureCollection(features)

with open('nextdoor_neighborhoods.geojson', 'w') as f:
   dump(feature_collection, f)

## Analysis and Visualization

### Viz 1:

In [ ]:
attribute_columns = ['State', 'County', 'City', 'Neighborhood', "Nextdoor ID", "Attribute", "Position"]
df_nextdoor_attributes = pd.DataFrame(columns=attribute_columns)

In [ ]:
def iterateOneNeighborhoodInterestActivity(currentRowIndex):
    global df_nextdoor_attributes
    entry = df_nextdoor_neighborhoods.iloc[currentRowIndex]
        
    attributes = entry["Attributes"]
    attributes = attributes.replace("[", "")
    attributes = attributes.replace("]", "")
    attributes = attributes.replace("'", "")
    attributes = attributes.split(", ")
    print(len(attributes))
    index = 0
    for attribute in attributes:
        print(attribute)
        df_nextdoor_attributes = df_nextdoor_attributes.append(
            {"State": entry['State'],
             "County": entry['County'],
             "City": entry['City'],
             "Neighborhood": entry['Neighborhood'],
             "Nextdoor ID": entry['Nextdoor ID'],
             "Attribute": attribute,
             "Position": index
             }, ignore_index=True)   
        index += 1

In [ ]:
iterateOneNeighborhoodInterestActivity(0) 

In [ ]:
currentIndex = 1
endIndex = len(df_nextdoor_neighborhoods) - 1
print("Number of Neighborhoods: " + str(len(df_nextdoor_neighborhoods)))
print("End Index: " , str(endIndex))
while True:
    print("Current Index: " + str(currentIndex))
    iterateOneNeighborhoodInterestActivity(currentIndex) 
    currentIndex += 1
    if currentIndex > endIndex:
        break

In [ ]:
df_nextdoor_attributes.tail()

In [ ]:
df_nextdoor_attributes.to_csv('nextdoor_attributes_master.csv', index=False)